* This script was written to extract alpha band parameters (individual alpha peak (IAF) and alpha power) from the .mat data. 
* The alpha band parameters were calculated using a custom Matlab script:
    * 'adapted_el_bandpower_on_detrended_psd_SSRI.m' to calculate parameters on detrended PSD

In [ ]:
import numpy as np
import pandas as pd
import scipy.io as spio
from openpyxl import Workbook

## Loading and Preparing the Data

In [ ]:
# load Matlab matrixes, extract parameters
# update the paths to the files accordingly
iaf_mat = spio.loadmat('')['iaf']
power_mat = spio.loadmat('')['broadpow']
subj_id_mat = spio.loadmat('')['ID_all'][0]
channels_mat = spio.loadmat('')['chan_label'][0]

**Creating lists to store IDs and channel names**

In [ ]:
names = []
for x in range(len(subj_id_mat)):
    name = subj_id_mat[x][0]
    names.append(name)

channels = []
for y in range(len(channels_mat)):
    channel = channels_mat[y][0]
    channels.append(channel)

**Converting lists to a numpy arrays and creating dataframes with parameter values with modified index values**

In [ ]:
iaf = np.array(iaf_mat)
power = np.array(power_mat)

iaf_ch_id = pd.DataFrame(iaf, columns=channels, index=[int(''.join(filter(str.isdigit, name))) for name in names])
power_ch_id = pd.DataFrame(power, columns=channels, index=[int(''.join(filter(str.isdigit, name))) for name in names])

**Adding the 'group' column based on ID values**

In [ ]:
# Reset the index and rename the column
iaf_ch_id = iaf_ch_id.reset_index().rename(columns={'index': 'id'})
power_ch_id = power_ch_id.reset_index().rename(columns={'index': 'id'})


# Create a mapping dictionary for the 'group' column
group_mapping = {
    (0, 100): 'OC',
    (100, 200): 'IUD',
    (200, 300): 'NCF',
    (300, 400): 'NCL',
    (400, float('inf')): 'M',
}

# Add the 'group' column 
iaf_ch_id.insert(1, 'group', pd.cut(iaf_ch_id['id'], bins=[0, 100, 200, 300, 400, float('inf')], labels=['OC', 'IUD', 'NCF', 'NCL', 'M']))
power_ch_id.insert(1, 'group', pd.cut(power_ch_id['id'], bins=[0, 100, 200, 300, 400, float('inf')], labels=['OC', 'IUD', 'NCF', 'NCL', 'M']))

## Assessing NaN Values Across Subjects and Channels

* NaN value threshold was established at 31 channels, constituting 50% of total channels.
* Subjects with data showing more than 31 missing alpha peaks were excluded from the analysis.

In [ ]:
nan_threshold = 31

# Initialize empty lists for filtered data
filtered_data_iaf = []
filtered_data_pw = []
subjects_exceeding_nan = []

for (iaf_row, pw_row) in zip(iaf_ch_id.iterrows(), power_ch_id.iterrows()):
    iaf_index, iaf_data = iaf_row
    pw_index, pw_data = pw_row
    
    # Assuming 'id' is the same for both iaf and power datasets
    subject_id = iaf_data['id']

    # Count NaNs (using either iaf or power, as NaN count is the same)
    nan_count = iaf_data.isnull().sum()

    # Filter both iaf and power data based on the NaN count
    if nan_count <= nan_threshold:
        filtered_data_iaf.append(iaf_data)
        filtered_data_pw.append(pw_data)
    else:
        subjects_exceeding_nan.append((subject_id, nan_count))

# Convert lists to dataframes
filtered_iaf = pd.DataFrame(filtered_data_iaf)
filtered_power = pd.DataFrame(filtered_data_pw)

# Print subjects with more than 31 NaN values
print("Subjects with more than 31 NaN values:")
for subject, nan_count in subjects_exceeding_nan:
    print(f"Subject ID: {subject}, NaN count: {nan_count}")


## Extracting Data From the Specific Channel(s) 
The folowing code is to:
* Extract the data from specific channel
* Calculate median parameter values for each group
* Save the data to Excel

In [ ]:
# Specify the channel
channel = 'Pz'

# power and iaf from a specified channel(s) for each subject
power_pz = filtered_power[['id', 'group', channel]]
iaf_pz = filtered_iaf[['id', 'group', channel]]

In [ ]:
data_frames = [
    ('iaf_Pz', iaf_pz),
    ('power_pz', power_pz),
    ('iaf_unfiltered_all_chans', iaf_ch_id),
    ('power_unfiltered_all_chans', power_ch_id),
    ('iaf_each_ch_sbj_removed_subj', filtered_iaf),
    ('power_each_ch_sbj_removed_subj', filtered_power),
]

with pd.ExcelWriter('iaf_power.xlsx') as writer:
    for sheet_name, data_frame in data_frames:
        data_frame.to_excel(writer, sheet_name=sheet_name, index=False)